# Carga de librerias

In [1]:
from pymongo import MongoClient
import time
from datetime import datetime, timedelta

client = MongoClient('localhost', 27017)
db = client["test_mongo"]

# Actualizaciones a la base de datos con 1 millón de registros y todos los campos indexados

En esta sección se revisará la operación de actualización para la colección de 1 millón de documentos ya indexados

In [15]:
coll = db["extracto_cuenta_1m2"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_1': {'v': 2, 'key': [('nota', 1)]}}

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [3]:
def actualizacion_fecha_monto(start_date, end_date, monto):
    from_date_str = start_date + "T00:00:00+00:00"
    to_date_str = end_date + "T23:59:59+00:00"
    
    criteria = {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    
    new = {
        "$set":
            {
                "monto": monto, 
                "nota": "Monto actualizado por fecha"
            }
        }

    return coll.update_many(criteria, new)

In [5]:
start_date = "2021-08-01"
end_date = "2021-08-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1152 documentos
 Actualización realizada en un tiempo de 0:00:45.321720


In [6]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db340dfb1bbbb1b844d21'), 'fecha': '2021-08-02 01:25:37', 'agencia': 'Santa Cruz', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 8837884.56, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db340dfb1bbbb1b8365b0'), 'fecha': '2021-08-02 02:08:28', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Deposito a cuenta', 'saldo': 535334.66, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db343dfb1bbbb1b8c7fba'), 'fecha': '2021-08-02 02:09:28', 'agencia': 'La Paz', 'monto': 7890, 'descripcion': 'Pago recibo de luz', 'saldo': 6480861.49, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db33fdfb1bbbb1b80a223'), 'fecha': '2021-08-02 02:40:03', 'agencia': 'Pando', 'monto': 7890, 'descripcion': 'Pago recibo de gas', 'saldo': 3829466.48, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db341dfb1bbbb1b866eef'), 'fecha': '2021-08-02 03:13:41', 'agencia': 'Beni', 'monto': 7890, 'descripcion': 'Pago en interés', 

## Actualización 2: Nota por descripción y fecha

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [4]:
def actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto):
    from_date_str = start_date + "T00:00:00+00:00"
    to_date_str = end_date + "T23:59:59+00:00"
    
    criteria = {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
        }
    
    new = {
        "$set":
            {
                "monto": monto, 
                "nota": "Monto actualizado por descripción"
            }
        }

    return coll.update_many(criteria, new)

In [8]:
start_date = "1990-05-14"
end_date = "1990-05-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 26 documentos
 Actualización realizada en un tiempo de 0:00:06.450380


In [9]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db33edfb1bbbb1b7ec963'), 'fecha': '1990-05-15 07:20:27', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5256702.47, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db33edfb1bbbb1b7dfeef'), 'fecha': '1990-05-15 10:54:20', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 9195278.66, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db341dfb1bbbb1b8665cc'), 'fecha': '1990-05-16 13:02:45', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1492747.54, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db33fdfb1bbbb1b80822e'), 'fecha': '1990-05-17 02:27:40', 'agencia': 'La Paz', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 6594102.26, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db342dfb1bbbb1b8ace52'), 'fecha': '1990-05-17 07:16:15', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [5]:
def actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion):

    criteria = {
        "$and":
            [
                {
                    "monto": {"$lt": monto}
                },
                {
                    "descripcion": descripcion
                },
                {
                    "$or":
                        [
                            {
                                "agencia": agencia1
                            },
                            {
                                "agencia": agencia2
                            }  
                        ]
                }
            ]
        }

    new = {
        "$set":
            {
                "saldo": saldo, 
                "nota": "Saldo actualizado por monto y agencias"
            }
        }

    return coll.update_many(criteria, new)

In [38]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 34 documentos
 Actualización realizada en un tiempo de 0:00:01.197947


In [39]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db33edfb1bbbb1b7d6de2'), 'fecha': '1973-12-30 15:18:51', 'agencia': 'La Paz', 'monto': 1090.72, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33edfb1bbbb1b7fa0f8'), 'fecha': '1992-09-23 08:51:16', 'agencia': 'Santa Cruz', 'monto': 1183.89, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33edfb1bbbb1b7faa61'), 'fecha': '2019-05-05 09:15:43', 'agencia': 'Santa Cruz', 'monto': 1100.83, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33edfb1bbbb1b80004d'), 'fecha': '1964-02-17 19:22:34', 'agencia': 'La Paz', 'monto': 1803.68, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db33fdfb1bbbb1b807973'), 'fecha': '1997-01-05 16:54:23', 'agencia': 'Santa Cruz', 'monto': 1687.35, 'descripcion': 'Interes

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 45.3217 s      |
| Tiempo actualización 2      | 6.4504 s       |
| Tiempo actualización 3      | 1.1979 s       |

-----------------

# Actualizaciones a la base de datos con 10 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos ya indexados

In [16]:
coll = db["extracto_cuenta_10m2"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_1': {'v': 2, 'key': [('nota', 1)]}}

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [42]:
start_date = "2021-12-01"
end_date = "2021-12-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 11439 documentos
 Actualización realizada en un tiempo de 0:04:33.459410


In [43]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db08fdfb1bba8ec4915df'), 'fecha': '2021-12-02 00:00:54', 'agencia': 'Tarija', 'monto': 7890, 'descripcion': 'Pago de cheque compensado', 'saldo': 7895316.3, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db05edfb1bba8ec2b16dd'), 'fecha': '2021-12-02 00:01:56', 'agencia': 'Santa Cruz', 'monto': 7890, 'descripcion': 'Pago recibo de agua', 'saldo': 1212965.69, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db0a9dfb1bba8ec5a1379'), 'fecha': '2021-12-02 00:04:40', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'Deposito a cuenta', 'saldo': 1100613.5, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db013dfb1bba8ecfac0ce'), 'fecha': '2021-12-02 00:05:00', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Pago recibo de gas', 'saldo': 5367940.51, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636db0abdfb1bba8ec5fa176'), 'fecha': '2021-12-02 00:09:02', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'DEB.CTA.P/C.INT

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [46]:
start_date = "1980-01-14"
end_date = "1980-04-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1241 documentos
 Actualización realizada en un tiempo de 0:04:00.600043


In [47]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db0a9dfb1bba8ec58cd80'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db017dfb1bba8ec01cc35'), 'fecha': '1980-01-15 02:24:13', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 1356620.84, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db092dfb1bba8ec4ecb6e'), 'fecha': '1980-01-15 02:24:49', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 215898.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636db0abdfb1bba8ec600b09'), 'fecha': '1980-01-15 03:49:51', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1483142.98, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636dafe6dfb1bba8ecdcb513'), 'fecha': '1980-01-15 04:25:00', 'agencia': 'Cochabamba', 'monto': 10.5, 'de

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [48]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 423 documentos
 Actualización realizada en un tiempo de 0:02:20.916705


In [49]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636db0a9dfb1bba8ec5833e4'), 'fecha': '1960-07-09 07:48:09', 'agencia': 'Santa Cruz', 'monto': 12.03, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636dafe9dfb1bba8ece2d0b0'), 'fecha': '2018-04-01 20:25:14', 'agencia': 'La Paz', 'monto': 16.2, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db0abdfb1bba8ec5f5b09'), 'fecha': '1964-04-12 18:53:53', 'agencia': 'Santa Cruz', 'monto': 19.62, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636db02edfb1bba8ec0b2d76'), 'fecha': '1956-04-14 01:32:42', 'agencia': 'La Paz', 'monto': 20.34, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636dafe5dfb1bba8ecda6e7e'), 'fecha': '1964-09-10 17:22:52', 'agencia': 'Santa Cruz', 'monto': 21.98, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido  |
|-----------------------------|-----------------|
| Tiempo actualización 1      | 4 min 33.4594 s |
| Tiempo actualización 2      | 4 min 0.6004 s  |
| Tiempo actualización 3      | 2 min 20.9167 s |

--------------------------

# Actualizaciones a la base de datos con 30 millones de registros y todos los campos indexados

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos ya indexados

In [17]:
coll = db["extracto_cuenta_30m"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_1': {'v': 2, 'key': [('nota', 1)]}}

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [21]:
start_date = "2009-12-01"
end_date = "2009-12-02"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 244 documentos
 Actualización realizada en un tiempo de 0:01:15.676908


In [60]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('636d40b3dfb1bb4dd6d924d5'), 'fecha': '2020-12-02 00:00:14', 'agencia': 'Tarija', 'monto': 7890, 'descripcion': 'Pago de nómina', 'saldo': 458926.79, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d3f87dfb1bb4dd6074ee9'), 'fecha': '2020-12-02 00:01:05', 'agencia': 'Chuquisaca', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 5416583.29, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d40b1dfb1bb4dd6d319fa'), 'fecha': '2020-12-02 00:01:15', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Apertura de cuenta corriente', 'saldo': 1716273.52, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d4126dfb1bb4dd620a662'), 'fecha': '2020-12-02 00:02:58', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'Reintegro cajero automático', 'saldo': 1545910.97, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636d41c3dfb1bb4dd67be379'), 'fecha': '2020-12-02 00:05:49', 'agencia': 'Cochabamba', 'monto': 7890, 'descri

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [23]:
start_date = "1981-01-14"
end_date = "1981-01-15"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 67 documentos
 Actualización realizada en un tiempo de 0:01:31.543552


In [24]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636d3fcfdfb1bb4dd6397ac5'), 'fecha': '1981-01-15 00:18:24', 'agencia': 'Pando', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 2380795.98, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d3ffddfb1bb4dd6610cf7'), 'fecha': '1981-01-15 01:09:39', 'agencia': 'Cochabamba', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5597202.33, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d410cdfb1bb4dd60ccdb0'), 'fecha': '1981-01-15 01:26:24', 'agencia': 'Santa Cruz', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 2122363.13, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d41addfb1bb4dd66a7b51'), 'fecha': '1981-01-15 01:30:10', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1337967.17, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('636d403edfb1bb4dd6815b1d'), 'fecha': '1981-01-15 02:35:54', 'agencia': 'Oruro', 'monto': 10.5, 'descrip

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [64]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 100

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 68 documentos
 Actualización realizada en un tiempo de 0:00:31.925104


In [65]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('636d3fe5dfb1bb4dd64a51dc'), 'fecha': '1960-07-09 07:48:09', 'agencia': 'Santa Cruz', 'monto': 12.03, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d3f31dfb1bb4dd6d4eea8'), 'fecha': '2018-04-01 20:25:14', 'agencia': 'La Paz', 'monto': 16.2, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d40f7dfb1bb4dd6fe4cef'), 'fecha': '1977-10-03 06:06:13', 'agencia': 'Santa Cruz', 'monto': 18.13, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d3fe7dfb1bb4dd6517901'), 'fecha': '1964-04-12 18:53:53', 'agencia': 'Santa Cruz', 'monto': 19.62, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('636d3f72dfb1bb4dd6fd4b6d'), 'fecha': '1956-04-14 01:32:42', 'agencia': 'La Paz', 'monto': 20.34, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido  |
|-----------------------------|-----------------|
| Tiempo actualización 1      | 1 min 15.6769 s |
| Tiempo actualización 2      | 1 min 31.5435 s |
| Tiempo actualización 3      | 31.9251 s       |

----------------

# Actualizaciones a la base de datos con 10 millones de registros y solo tres campos indexados (fecha, descripción, nota)

En esta sección se revisará la operación de búsqueda para la colección de 10 millones de documentos ya indexados en tres campos: fecha, descripción, nota.

In [6]:
coll = db["extracto_cuenta_10m_3idx"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('nota', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [7]:
start_date = "2009-12-01"
end_date = "2009-12-05"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1623 documentos
 Actualización realizada en un tiempo de 0:07:44.294338


In [8]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('63740c90dfb1bb283709908b'), 'fecha': '2009-12-02 00:01:51', 'agencia': 'Tarija', 'monto': 7890, 'descripcion': 'Pago recibo de gas', 'saldo': 6098102.06, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63740c1bdfb1bb2837b96572'), 'fecha': '2009-12-02 00:02:07', 'agencia': 'Santa Cruz', 'monto': 7890, 'descripcion': 'Pago de nómina', 'saldo': 9166286.86, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63740c8ddfb1bb2837028677'), 'fecha': '2009-12-02 00:15:46', 'agencia': 'Pando', 'monto': 7890, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 1937050.57, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63740ccedfb1bb283731f4ba'), 'fecha': '2009-12-02 00:26:34', 'agencia': 'Santa Cruz', 'monto': 7890, 'descripcion': 'Pago en interés', 'saldo': 5757902.92, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63740c77dfb1bb2837f16f22'), 'fecha': '2009-12-02 00:26:44', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'AB.CTA.P/MOVIL', 'sa

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [11]:
start_date = "1980-01-14"
end_date = "1980-01-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto) 
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 271 documentos
 Actualización realizada en un tiempo de 0:02:46.527624


In [12]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('63740ce3dfb1bb2837416e0e'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63740c66dfb1bb2837ea6cc2'), 'fecha': '1980-01-15 02:24:13', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 1356620.84, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63740cd0dfb1bb2837376bfc'), 'fecha': '1980-01-15 02:24:49', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 215898.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63740ce6dfb1bb283748ab97'), 'fecha': '1980-01-15 03:49:51', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1483142.98, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63740c35dfb1bb2837c555a1'), 'fecha': '1980-01-15 04:25:00', 'agencia': 'Cochabamba', 'monto': 10.5, 'de

## Actualización 3: Saldo por agencias o monto

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [13]:
agencia1 = "Oruro"
agencia2 = "Potosi"
descripcion = 'Intereses'
saldo = 1234.5
monto = 200

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 20 documentos
 Actualización realizada en un tiempo de 0:03:18.235317


In [14]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('63740c1bdfb1bb2837b98932'), 'fecha': '2017-11-01 13:08:13', 'agencia': 'Oruro', 'monto': 126.87, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740c34dfb1bb2837c2d861'), 'fecha': '1955-09-03 15:58:54', 'agencia': 'Oruro', 'monto': 172.05, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740c38dfb1bb2837ca0c6e'), 'fecha': '1969-04-27 15:44:04', 'agencia': 'Oruro', 'monto': 138.31, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740c38dfb1bb2837cb6162'), 'fecha': '1954-09-28 14:45:39', 'agencia': 'Oruro', 'monto': 70.05, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740c38dfb1bb2837cb7ae5'), 'fecha': '1984-08-01 05:16:38', 'agencia': 'Oruro', 'monto': 40.56, 'descripcion': 'Intereses', 'saldo': 1234.5, 'n

| Tabla resumen de resultados | Valor obtenido   |
|-----------------------------|------------------|
| Tiempo actualización 1      | 7 min 44.2943 s  |
| Tiempo actualización 2      | 2 min 46.5276 s  |
| Tiempo actualización 3      | 3 min 18.2353 s  |

---------------------

# Actualizaciones a la base de datos con 30 millones de registros y solo tres campos indexados (fecha, descripción, nota)

En esta sección se revisará la operación de búsqueda para la colección de 30 millones de documentos ya indexados en tres campos: fecha, descripción, nota.

In [15]:
coll = db["extracto_cuenta_30m_3idx"]
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'descripcion_1': {'v': 2, 'key': [('descripcion', 1)]},
 'nota_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('nota', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [16]:
start_date = "2008-12-01"
end_date = "2008-12-02"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1131 documentos
 Actualización realizada en un tiempo de 0:06:06.351776


In [17]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
        }
    )

for result in results[:10]:
    print(result)

{'_id': ObjectId('63740fc2dfb1bb38874963f0'), 'fecha': '2008-12-02 00:01:46', 'agencia': 'Cochabamba', 'monto': 7890, 'descripcion': 'Reintegro cajero automático', 'saldo': 7070822.19, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('637410cfdfb1bb38870e655a'), 'fecha': '2008-12-02 00:02:09', 'agencia': 'La Paz', 'monto': 7890, 'descripcion': 'Transferencia de fondos de ahorros', 'saldo': 7788483.75, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('637410b6dfb1bb3887fa31a2'), 'fecha': '2008-12-02 00:02:34', 'agencia': 'Chuquisaca', 'monto': 7890, 'descripcion': 'Intereses', 'saldo': 7757672.48, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63741002dfb1bb388771e95f'), 'fecha': '2008-12-02 00:05:23', 'agencia': 'Oruro', 'monto': 7890, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 3623915.14, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('637410f2dfb1bb38872a5d24'), 'fecha': '2008-12-02 00:05:52', 'agencia': 'Santa Cruz', 'monto': 7890, 'descr

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [19]:
start_date = "1980-01-14"
end_date = "1980-01-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 807 documentos
 Actualización realizada en un tiempo de 0:08:30.047104


In [20]:
from_date_str = start_date + "T00:00:00+00:00"
to_date_str = end_date + "T23:59:59+00:00"

results = coll.find(
    {
        "$and":
            [
                {
                    "$or":
                    [
                        {
                            "descripcion": descripcion1
                        },
                        {
                            "descripcion": descripcion2
                        }
                    ]
                },
                {
                    "fecha":
                    {
                        "$gte": from_date_str,
                        "$lte": to_date_str
                    }
                }
            ]
            
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('6374111cdfb1bb3887527eac'), 'fecha': '1980-01-15 00:03:48', 'agencia': 'Santa Cruz', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 1191798.41, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('637410f7dfb1bb38873696fe'), 'fecha': '1980-01-15 00:16:58', 'agencia': 'Tarija', 'monto': 10.5, 'descripcion': 'Apertura de caja de ahorros', 'saldo': 4173801.78, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63740fc2dfb1bb38874aa3df'), 'fecha': '1980-01-15 00:24:34', 'agencia': 'Beni', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 5431030.18, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63741005dfb1bb3887792787'), 'fecha': '1980-01-15 01:06:53', 'agencia': 'Chuquisaca', 'monto': 10.5, 'descripcion': 'Aporte inversor', 'saldo': 8847975.29, 'nota': 'Monto actualizado por descripción'}
{'_id': ObjectId('63741092dfb1bb3887e52e56'), 'fecha': '1980-01-15 01:16:24', 'agencia': 'Cochabamba', 'monto': 10.5, 'descrip

## Actualización 3: Saldo por agencias o monto

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [22]:
agencia1 = "La Paz"
agencia2 = "Santa Cruz"
descripcion = 'Intereses'
saldo = 1234.5
monto = 200

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results.modified_count} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 137 documentos
 Actualización realizada en un tiempo de 0:13:40.891371


In [23]:
results = coll.find(
    {
    "$and":
        [
            {
                "monto": {"$lt": monto}
            },
            {
                "descripcion": descripcion
            },
            {
                "$or":
                    [
                        {
                            "agencia": agencia1
                        },
                        {
                            "agencia": agencia2
                        }  
                    ]
            }
        ]
    }
)

for result in results[:10]:
    print(result)

{'_id': ObjectId('63740eeedfb1bb3887bee07f'), 'fecha': '1955-01-30 00:27:19', 'agencia': 'Santa Cruz', 'monto': 155.16, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740eeedfb1bb3887c074a0'), 'fecha': '2019-01-07 04:12:30', 'agencia': 'Santa Cruz', 'monto': 115.47, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740ef0dfb1bb3887c3da66'), 'fecha': '2011-07-25 02:59:35', 'agencia': 'La Paz', 'monto': 154.42, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740ef0dfb1bb3887c43be2'), 'fecha': '1989-05-31 12:37:13', 'agencia': 'La Paz', 'monto': 60.51, 'descripcion': 'Intereses', 'saldo': 1234.5, 'nota': 'Saldo actualizado por monto y agencias'}
{'_id': ObjectId('63740ef1dfb1bb3887c70dad'), 'fecha': '1977-11-10 05:02:53', 'agencia': 'La Paz', 'monto': 74.07, 'descripcion': 'Intereses', 'saldo

| Tabla resumen de resultados | Valor obtenido  |
|-----------------------------|-----------------|
| Tiempo actualización 1      | 6 min 6.3517 s  |
| Tiempo actualización 2      | 8 min 30.0471 s |
| Tiempo actualización 3      | 13 min 40.8913 s |